# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

## Chain Class 基类

类继承关系：

```
Chain --> <name>Chain  # Examples: LLMChain, MapReduceChain, RouterChain
```

**代码实现：https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/base.py**

```python
# 定义一个名为Chain的基础类
class Chain(Serializable, Runnable[Dict[str, Any], Dict[str, Any]], ABC):
    """为创建结构化的组件调用序列的抽象基类。
    
    链应该用来编码对组件的一系列调用，如模型、文档检索器、其他链等，并为此序列提供一个简单的接口。
    
    Chain接口使创建应用程序变得容易，这些应用程序是：
    - 有状态的：给任何Chain添加Memory可以使它具有状态，
    - 可观察的：向Chain传递Callbacks来执行额外的功能，如记录，这在主要的组件调用序列之外，
    - 可组合的：Chain API足够灵活，可以轻松地将Chains与其他组件结合起来，包括其他Chains。
    
    链公开的主要方法是：
    - `__call__`：链是可以调用的。`__call__`方法是执行Chain的主要方式。它将输入作为一个字典接收，并返回一个字典输出。
    - `run`：一个方便的方法，它以args/kwargs的形式接收输入，并将输出作为字符串或对象返回。这种方法只能用于一部分链，不能像`__call__`那样返回丰富的输出。
    """

    # 调用链
    def invoke(
        self, input: Dict[str, Any], config: Optional[runnableConfig] = None
    ) -> Dict[str, Any]:
        """传统调用方法。"""
        return self(input, **(config or {}))

    # 链的记忆，保存状态和变量
    memory: Optional[BaseMemory] = None
    """可选的内存对象，默认为None。
    内存是一个在每个链的开始和结束时被调用的类。在开始时，内存加载变量并在链中传递它们。在结束时，它保存任何返回的变量。
    有许多不同类型的内存，请查看内存文档以获取完整的目录。"""

    # 回调，可能用于链的某些操作或事件。
    callbacks: Callbacks = Field(default=None, exclude=True)
    """可选的回调处理程序列表（或回调管理器）。默认为None。
    在对链的调用的生命周期中，从on_chain_start开始，到on_chain_end或on_chain_error结束，都会调用回调处理程序。
    每个自定义链可以选择调用额外的回调方法，详细信息请参见Callback文档。"""

    # 是否详细输出模式
    verbose: bool = Field(default_factory=_get_verbosity)
    """是否以详细模式运行。在详细模式下，一些中间日志将打印到控制台。默认值为`langchain.verbose`。"""

    # 与链关联的标签
    tags: Optional[List[str]] = None
    """与链关联的可选标签列表，默认为None。
    这些标签将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""

    # 与链关联的元数据
    metadata: Optional[Dict[str, Any]] = None
    """与链关联的可选元数据，默认为None。
    这些元数据将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""
```

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

In [13]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.9, max_tokens=500)

In [14]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="给制造{product}的有限公司取10个好名字，并给出完整的公司名称",
)

In [15]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
print(chain.invoke({
    'product': "性能卓越的GPU"
    }))

{'product': '性能卓越的GPU', 'text': '\n\n1. 晶灵科技（CrystalTech Inc.）\n2. 宇宙芯片（CosmoChip Co.）\n3. 火箭图形（RocketGraphics Ltd.）\n4. 神经芯片（NeuroChip Corp.）\n5. 火山晶片（VolcanoPixel Inc.）\n6. 星际引擎（InterstellarEngine Ltd.）\n7. 光速加速器（LightSpeed Accelerator Co.）\n8. 极光科技（AuroraTech Inc.）\n9. 超晶科技（UltraCrystal Technologies Ltd.）\n10. 量子显卡（QuantumGPUs Co.）'}


In [16]:
chain.verbose =True

In [17]:
chain.verbose

True

In [18]:
print(chain.invoke({
    'product': "性能卓越的GPU"
    }))



> Entering new LLMChain chain...
Prompt after formatting:
给制造性能卓越的GPU的有限公司取10个好名字，并给出完整的公司名称

> Finished chain.
{'product': '性能卓越的GPU', 'text': '\n\n1. 雷神计算机技术有限公司(Ragnarok Computing Technology Co., Ltd.)\n2. 爱普希测试设备有限公司(Apex Test Equipment Co., Ltd.)\n3. 火箭科技图形处理器有限公司(Rocket Technology Graphics Inc.)\n4. 新星晶体技术有限公司(Nova Crystal Technologies Co., Ltd.)\n5. 强力顶点处理公司(Power Vertex Processing Co., Ltd.)\n6. 极速像素公司(Extreme Pixel Inc.)\n7. 光影计算引擎有限公司(Lightworks Computing Engine Co., Ltd.)\n8. 纳米渲染系统有限公司(Nanorendering Systems Co., Ltd.)\n9. 奇迹绘图芯片有限公司(Miracle Graphics Chips Co., Ltd.)\n10. 竞技电子保卫公司(Agile Electronic Defense Co., Ltd.)'}


## Sequential Chain

串联式调用语言模型（将一个调用的输出作为另一个调用的输入）。

顺序链（Sequential Chain ）允许用户连接多个链并将它们组合成执行特定场景的流水线（Pipeline）。有两种类型的顺序链：

- SimpleSequentialChain：最简单形式的顺序链，每个步骤都具有单一输入/输出，并且一个步骤的输出是下一个步骤的输入。
- SequentialChain：更通用形式的顺序链，允许多个输入/输出。

### 使用 SimpleSequentialChain 实现戏剧摘要和评论（单输入/单输出）

![](../images/simple_sequential_chain_0.png)

In [19]:
# 这是一个 LLMChain，用于根据剧目的标题撰写简介。

llm = OpenAI(temperature=0.7, max_tokens=1000)

template = """你是一位剧作家。根据戏剧的标题，你的任务是为该标题写一个简介。

标题：{title}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

In [20]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。
# llm = OpenAI(temperature=0.7, max_tokens=1000)
template = """你是《纽约时报》的戏剧评论家。根据剧情简介，你的工作是为该剧撰写一篇评论。

剧情简介：
{synopsis}

以下是来自《纽约时报》戏剧评论家对上述剧目的评论："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

![](../images/simple_sequential_chain_1.png)

In [21]:
# 这是一个SimpleSequentialChain，按顺序运行这两个链
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)

In [11]:
review = overall_chain.invoke("三体人不是无法战胜的")



> Entering new SimpleSequentialChain chain...


《三体人不是无法战胜的》是一部关于宇宙中最强大的种族——三体人的故事。在这个宇宙中，三体人被认为是无法战胜的，因为他们拥有超越常人的科技和力量。然而，一位普通人类的出现改变了一切。他发现了三体人的弱点，并决心与他们抗争，为人类的自由而战。在与三体人的殊死搏斗中，他发现三体人并非不可战胜，而是因为他们的内心缺乏爱和同情。最终，普通人类的勇气和坚持，打破了三体人的统治，带来了和平与和谐的新时代。这部戏剧探讨了人类与外星种族的关系，以及爱和同情的力量对战争的影响。它将带领观众进入一个惊险刺激的宇宙冒险，同时也给人们带来思考和启发。


《三体人不是无法战胜的》是一部充满惊险和启发的戏剧作品。它通过一个普通人类与宇宙中最强大的种族——三体人的对抗，探讨了人类与外星种族的关系，以及爱和同情的力量对战争的影响。

这部作品精彩地展现了三体人的科技与力量，以及他们对人类的统治。观众们将被带入一个充满刺激的宇宙冒险，与主角一起感受战斗的紧张与惊险。同时，剧中也深刻地探讨了三体人的弱点，以及人类如何通过勇气和坚持最终战胜他们。

最令人感动的是，剧中展现了爱和同情的力量。主角发现，三体人之所以无法战胜，并不是因为他们的科技和力量，而是因为他们内心缺乏爱和同情。这也引发了人们对战争的思考，让观众们深刻地反思爱和同情对于和平与和谐的重要性。

整部戏剧的制作也十分精彩，舞台设计、服装和道具都充分展现了宇宙的奇幻和未来的科技感。演员们的表演也令人印象深刻，他们生动地诠释了各个角色的复杂心理和情感。

总的来说，《三体人不是无法战胜的》是一部令人难忘的戏剧作品。它不仅带给观众惊险的冒险和刺激的战斗，更重要的是，它让我们思考人类与外星种族的关系，以及爱和同情的力量对于和平与和谐的重要性。这部作品将在观众心中留下深刻的印象，值得一看。

> Finished chain.


In [22]:
review = overall_chain.invoke("星球大战第九季")



> Entering new SimpleSequentialChain chain...


《星球大战第九季》是一部充满惊险刺激的科幻戏剧，它继承了前八部《星球大战》的传奇故事，继续讲述银河系中的光明与黑暗的斗争。在这一季中，反抗军和帝国军团都重新集结，双方将展开一场决定性的大战。同时，新的恶势力也浮出水面，试图破坏双方的平衡。在这个紧张的局势下，主角们将面临更艰巨的挑战，不仅要保护自己所珍视的人和事，还要为整个银河系的命运而奋斗。精彩的战斗场面、悬念迭起的剧情和充满想象力的特效将让观众沉浸在这个宏大的星战世界中，感受到不同于以往的震撼和感动。《星球大战第九季》将带领观众探索更多关于力量、命运和自我成长的深层次主题，同时也会为这部经典系列留下最精彩的结局。敬请期待。


《星球大战第九季》延续了这部经典系列的传奇故事，为观众带来了一场充满惊险刺激的科幻戏剧。剧中的战斗场面精彩绝伦，特效惊艳，令人沉浸在这个宏大的星战世界中。同时，剧情的悬念和紧张氛围也让观众屏息以待，难以释怀。

除了精彩的战斗场面和悬念的剧情，本季还将带领观众探索更深层次的主题，如力量、命运和自我成长。这些主题将为这部经典系列增添更多的意义和内涵，也让观众在欣赏戏剧的同时，思考人类的本质和价值观。

最令人期待的是，这部戏剧将为这部经典系列留下最精彩的结局。观众们将会在最后一集中见证主角们的成长和命运的转变，也为这部经典系列画上完美的句号。

总的来说，《星球大战第九季》将为观众带来一场视听盛宴，同时也会让观众在感受震撼和刺激的同时，思考更多的人生意义。这部戏剧绝对值得一看，敬请期待。

> Finished chain.


### 使用 SequentialChain 实现戏剧摘要和评论（多输入/多输出）

![](../images/sequential_chain_0.png)

In [27]:
# # 这是一个 LLMChain，根据剧名和设定的时代来撰写剧情简介。
llm = OpenAI(temperature=.7) # by wy：model默认是Text-embedding-ada-002-v2，max_tokens默认是256？
template = """你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：{title}
时代：{era}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title", "era"], template=template)
# output_key
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis", verbose=True)

In [28]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。

template = """你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：
{synopsis}

来自《纽约时报》戏剧评论家对上述剧目的评价："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review", verbose=True)

In [29]:
from langchain.chains import SequentialChain

m_overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [30]:
m_overall_chain.invoke({"title":"三体人不是无法战胜的", "era": "二十一世纪的新中国"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：三体人不是无法战胜的
时代：二十一世纪的新中国
剧作家：以下是对上述戏剧的简介：

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：


在新中国二十一世纪，人类迎来了一个全新的时代。科技的飞速发展让人类的生活变得更加舒适和便利，但同时也带来了前所未有的挑战。在这个时代，人类发现了地球以外的宇宙世界，其中最引人注目的就是三体人。

三体人是一种拥有高度智能和超强进化能力的外星种族，他们的科技水平远远超过人类，甚至可以操控整个宇宙。在人类与三体人的第一次接触中，人类发现自己并不是宇宙中唯一的智慧生命。

人类对三体人的存在感到恐惧和不安，担心他们会带来灾难和毁灭。但是，一位勇敢的科

来自《纽约时报》戏剧评论家对上述剧目的评价：

> Finished chain.

> Finished chain.


{'title': '三体人不是无法战胜的',
 'era': '二十一世纪的新中国',
 'synopsis': '\n\n在新中国二十一世纪，人类迎来了一个全新的时代。科技的飞速发展让人类的生活变得更加舒适和便利，但同时也带来了前所未有的挑战。在这个时代，人类发现了地球以外的宇宙世界，其中最引人注目的就是三体人。\n\n三体人是一种拥有高度智能和超强进化能力的外星种族，他们的科技水平远远超过人类，甚至可以操控整个宇宙。在人类与三体人的第一次接触中，人类发现自己并不是宇宙中唯一的智慧生命。\n\n人类对三体人的存在感到恐惧和不安，担心他们会带来灾难和毁灭。但是，一位勇敢的科',
 'review': '\n\n《三体人》这部戏剧令人印象深刻，它不仅仅是一部科幻作品，更是对人类与宇宙关系的深刻思考与探索。\n\n剧中的设定非常具有想象力，将现实与虚构巧妙地结合，让观众仿佛置身于一个全新的世界。科技的发展带来的便利和挑战也被巧妙地融入剧情中，令人不禁思考人类未来的发展方向。\n\n而最引人注目的，当然是三体人这一外星种族。他们拥有超强的智能和进化能力，几乎可以操控整个宇宙。与此同时，他们也成为人类的威胁，让人类感到恐惧和不安。这'}

### Homework

#### 使用 OutputParser 优化 overall_chain 输出格式，区分 synopsis_chain 和 review_chain 的结果